<img src="../img/UAX.png" width="300">

### Profesor: Jorge Calvo


## Autoencoders Variacionales (VAE)

Los autoencoders variacionales (VAE) son una extensión de los autoencoders convencionales que se utilizan para aprender una representación latente de los datos de entrada. A diferencia de los autoencoders tradicionales, los VAE tienen una estructura probabilística que les permite generar nuevas muestras de datos similares a las del conjunto de entrenamiento. Esta característica los hace útiles en tareas como la generación de imágenes, síntesis de texto y otras aplicaciones donde se necesita la generación de nuevos datos similares a los ya existentes.

<img src="../img/vae.png" width="1000">

<img src="../img/z.png" width="600">



### 1. Arquitectura Básica

1. **Encoder**  
   - Toma una entrada \(x\) (por ejemplo, una imagen) y la proyecta a un espacio latente de baja dimensión.  
   - A diferencia del autoencoder clásico, el encoder no produce un solo vector latente, sino dos vectores:
   
     - $\boldsymbol{\mu}(x) $: el **vector de medias**.
     - $ \boldsymbol{\sigma}(x) $: el **vector de desviaciones estándar** (o log-varianzas).

2. **Cuello de Botella (Latent Space)**  
   - En lugar de un punto determinístico, representamos cada entrada como una **distribución gaussiana** $ q(z|x) = \mathcal{N}\bigl(z;\,\boldsymbol{\mu}(x),\,\mathrm{diag}(\boldsymbol{\sigma}^2(x))\bigr)$.  
   - Para muestrear un punto $ z $ de esta distribución y permitir el paso de gradiente, aplicamos el **truco de reparametrización**:
     $
       z = \underbrace{\boldsymbol{\mu}(x)}_{\text{media}} + 
           \underbrace{\boldsymbol{\sigma}(x)}_{\text{desviación}} \;\odot\;
           \underbrace{\epsilon}_{\mathcal{N}(0,I)}
     $
   - Aquí $\epsilon\sim\mathcal{N}(0,I)$ añade estocasticidad, pero la parte diferenciable (medias y desviaciones) permite backpropagation.

3. **Decoder**  
   - Toma la muestra $ z $ y genera una reconstrucción $ \hat{x}$.  
   - Aprende la función inversa que, idealmente, recupera la distribución original de \(x\).

---

### 2. ¿Por qué Media y Desviación en el Bottleneck?

- **Media $\mu $**:  
  Describe el “centro” de la distribución latente para cada $x $.  
- **Desviación $\sigma $**:  
  Indica cuánta variabilidad permitimos alrededor de esa media.  

Esta parametrización probabilística aporta dos beneficios clave:

1. **Regularización Suave**: Evita que diferentes entradas se colapsen a un mismo punto latente; en lugar de eso, cada $x$ ocupa una pequeña “nube” en el espacio latente.  
2. **Generación de Muestras Nuevas**: Podemos muestrear distintos $\epsilon$ y generar múltiples $z$ para un mismo $x$, o incluso muestrear de la distribución estándar $\mathcal{N}(0,I)$ para crear datos completamente nuevos y coherentes.

---

### 3. Función de Pérdida Combinada

La función de pérdida de un VAE consta de dos términos:

1. **Reconstrucción** $\displaystyle \mathcal{L}_{\mathrm{rec}} = -\mathbb{E}_{q(z|x)}\bigl[\log p(x|z)\bigr]$
   — Mide cuán bien el decoder puede reconstruir $x$ a partir de $z$.  
   — Equivalente a la pérdida habitual de autoencoder (MSE, cross-entropy, …).

2. **Divergencia KL** $\displaystyle \mathcal{L}_{\mathrm{KL}} = D_{\mathrm{KL}}\bigl(q(z|x)\parallel p(z)\bigr)$ 
   — Mide cuánto difiere la distribución latente $q(z|x)$ de la **prior** $p(z)$ (normalmente \($\mathcal{N}(0,I)$)).  
   — Fórmula cerrada para Gaussianas:
   $
     D_{\mathrm{KL}}\bigl(\mathcal{N}(\mu,\sigma^2)\,\|\,\mathcal{N}(0,1)\bigr)
     = \tfrac12 \sum_{i=1}^d \Bigl(\mu_i^2 + \sigma_i^2 - \log\sigma_i^2 - 1\Bigr)
   $

El **objetivo total** que minimizamos es:
$
  \mathcal{L}_{\mathrm{VAE}}(x) 
  = \underbrace{\mathcal{L}_{\mathrm{rec}}}_{\text{Calidad de reconstrucción}}
  \;+\; 
  \underbrace{\mathcal{L}_{\mathrm{KL}}}_{\text{Regularización del latente}}
$
- El término KL fuerza a las medias $\mu(x)$ y desviaciones $\sigma(x)$ a aproximarse a $(0,1)$, evitando “sobreajuste” del espacio latente.  
- La reconstrucción asegura que la decodificación sea fiel a los datos originales.

---

### 4. Resumen Didáctico

1. **Encoder → $(\mu, \sigma)$**: Cada imagen $x$ se mapea a una **distribución** latente, no a un punto fijo.  
2. **Reparametrización**:  
   $
     z = \mu + \sigma \,\odot\, \epsilon \quad,\quad \epsilon\sim\mathcal{N}(0,I)
   $
   Permite pasar gradientes durante el muestreo.  
3. **Decoder → $\hat{x}$**: Reconstruye a partir de $z$.  
4. **Pérdida = Reconstrucción + KL**:  
   - **Reconstrucción** garantiza fidelidad.  
   - **KL** impone una distribución latente estándar, favoreciendo interpolación y generación de muestras.

Con este esquema tus alumnos pueden entender por qué **media** y **desviación** en el cuello de botella, y cómo la combinación de **reconstrucción** y **divergencia KL** produce un modelo probabilístico que **aprende** una representación latente continua, estructurada y generativa.



### Ejemplo Numérico Paso a Paso de un VAE


#### 1. Salidas del Encoder

Supongamos que el encoder, aplicado a una entrada $x$, produce:

- Vector de medias:
  $
    \boldsymbol{\mu} = [\,1.0,\;2.0\,]
  $
- Vector de log-varianzas:
  $
    \log \boldsymbol{\sigma}^2 = [\,-1.386,\;-3.219\,]
  $

---

#### 2. Cálculo de la desviación $\boldsymbol{\sigma}$

Recordemos que
$
  \sigma_i = \exp\!\bigl(\tfrac12 \log \sigma_i^2\bigr).
$

- Para $i=1$:
  $
    \log \sigma_1^2 = -1.386 
    \quad\Longrightarrow\quad
    \sigma_1 = \exp(-1.386/2) = \exp(-0.693) \approx 0.50
  $
- Para \(i=2\):
  $
    \log \sigma_2^2 = -3.219 
    \quad\Longrightarrow\quad
    \sigma_2 = \exp(-3.219/2) = \exp(-1.609) \approx 0.20
  $

Así,
$
  \boldsymbol{\sigma} = [\,0.5,\;0.2\,].
$

---

#### 3. Muestreo con el truco de reparametrización

Tomamos
$
  \epsilon \sim \mathcal{N}(0,I),
  \quad
  z = \boldsymbol{\mu} + \boldsymbol{\sigma} \,\odot\, \epsilon.
$

Si elegimos por ejemplo
$
  \epsilon = [\,1.0,\;-1.0\,],
$
entonces

$
  z_1 = 1.0 + 0.5 \times 1.0 = 1.5,
  \quad
  z_2 = 2.0 + 0.2 \times (-1.0) = 1.8,
$
y obtenemos
$
  \mathbf{z} = [\,1.5,\;1.8\,].
$

---

#### 4. Cálculo de la Divergencia KL

Para cada dimensión $i$, la contribución es

$
  D_{KL}\bigl(\mathcal{N}(\mu_i,\sigma_i^2)\,\|\;\mathcal{N}(0,1)\bigr)
  = \tfrac12\bigl(\mu_i^2 + \sigma_i^2 - \log \sigma_i^2 - 1\bigr).
$

- **Dimensión 1**:
  $
    \mu_1^2 = 1^2 = 1,\quad
    \sigma_1^2 = 0.25,\quad
    \log\sigma_1^2 = -1.386,
  $
  $
    d_1
    = \tfrac12\bigl(1 + 0.25 - (-1.386) - 1\bigr)
    = \tfrac12(1.25 + 1.386 - 1)
    = 0.818
  $

- **Dimensión 2**:
  $
    \mu_2^2 = 4,\quad
    \sigma_2^2 = 0.04,\quad
    \log\sigma_2^2 = -3.219,
  $
  $
    d_2
    = \tfrac12\bigl(4 + 0.04 - (-3.219) - 1\bigr)
    = \tfrac12(3.04 + 3.219)
    = 3.130
  $

Sumando ambas:
$
  D_{KL} = d_1 + d_2 \approx 0.818 + 3.130 = 3.948.
$

---

#### Resumen de Valores

- $\boldsymbol{\mu} = [1.0,\;2.0]$
- $\log \boldsymbol{\sigma}^2 = [-1.386,\;-3.219]$  
- $\boldsymbol{\sigma} = [0.5,\;0.2]$  
- $\epsilon = [1.0,\;-1.0]$  
- $\mathbf{z} = [1.5,\;1.8]$  
- $D_{KL} \approx 3.948$



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models, losses, backend as K
from tensorflow.keras.datasets import mnist
from keras.models import Model, load_model
import os

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Visualizar algunas imágenes de entrenamiento
fig, axs = plt.subplots(2, 5, figsize=(10, 4))
for i, ax in enumerate(axs.flat):
    ax.imshow(x_train[i], cmap='gray')
    ax.set_title(f"Label: {y_train[i]}")
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
latent_dim = 3

In [ ]:
#Encoder
encoder_input = tf.keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, (3, 3), padding="same", activation='relu')(encoder_input)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), padding="same", activation='relu')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)

### Codificación Probabilística:

El encoder toma una entrada x y produce dos salidas: la media (μ) y la varianza logarítmica (log(𝜎²))
Estas salidas definen una distribución gaussiana N(𝜇,𝜎²) en el espacio latente.
Un punto z en el espacio latente se obtiene muestreando de esta distribución.

In [ ]:
#Capa de muestreo
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

In [ ]:
#Decoder
decoder_input = layers.Input(shape=(latent_dim,))
x = layers.Dense(7*7*64, activation='relu')(decoder_input)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

In [ ]:
#Construir VAE
encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name="encoder")
decoder = models.Model(decoder_input, decoded, name="decoder")
vae_input = encoder_input
vae_output = decoder(z)
vae = models.Model(vae_input, vae_output, name="vae")

In [ ]:
plot_model(vae, to_file='./vae.png', show_shapes=True, show_layer_names=True, dpi=96)

### Pérdida del VAE:

La función de pérdida de un VAE tiene dos componentes:

* Pérdida de Reconstrucción: Mide qué tan bien el decoder puede reconstruir la entrada original a partir del punto muestreado en el espacio latente.

* Pérdida KL (Kullback-Leibler): Mide la divergencia entre la distribución aprendida por el encoder y una distribución gaussiana estándar. Esta regularización asegura que las representaciones latentes sean continuas y bien distribuidas.
La pérdida total se define como:


#### Perdida Total = Perdida Reconstrucción + Pérdida KL

 
$
\begin{equation}
\mathcal{L}_{\text{KL}} = -\frac{1}{2} \sum_{i=1}^{d} \left( 1 + \log(\sigma_i^2) - \mu_i^2 - \sigma_i^2 \right)
\end{equation}
$

<img src="../img/latent.png" width="1000">

<img src="../img/Kullback-Leibler.png" width="400">

https://www.europeanvalley.es/noticias/el-espacio-latente-en-la-ia/

In [ ]:
def normal_sample(x, mean, var):
    return ((1 / (np.sqrt(2 * np.pi * var))) * np.exp(-0.5 * (x - mean) ** 2 / var))

# Setting mean, variance and x
mean = [0.0, 0.0, 0.0, -2.0]
var = [0.2, 1.0, 5.0, 0.5]
x_range = np.arange(-5.0, 5.0, 0.05)
curves = []

# Computing distributions from each mean-var combiantions
for i in range(len(mean)):
    crv = [normal_sample(x, mean[i], var[i]) for x in x_range]
    curves.append(crv)

# Visualize curves
plt.figure(figsize=(8, 4))

for i in range(len(curves)):
    plt.plot(x_range, curves[i], 
             label=f"$\mu$={mean[i]}, $\sigma^2$={var[i]}")

plt.grid("on")
plt.legend()
plt.show()

$
f(x)= \frac{1}{\sigma\sqrt{2\pi}}\exp^{-\frac{(x-\mu)²}{2\sigma²}}
$

In [ ]:
#Función perdida

In [ ]:
#Entrenar


In [ ]:
#Visualizar resultados